Verification face

In [36]:
import os
import csv
import pickle
import numpy as np
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Iterable
import face_recognition

C:\Users\PC-09\Documents\Projects\face_recognition_svm\.venv\Lib\site-packages\face_recognition_models\__init__.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


In [37]:
# =========================
# CẤU HÌNH ĐƯỜNG DẪN
# =========================
ROOT_DIR = Path.cwd()
DATA_DIR = ROOT_DIR / "data"
LFW_DIR = DATA_DIR / "lfw-deepfunneled" / "lfw-deepfunneled"

In [39]:
# =========================
# CẤU HÌNH FACE PIPELINE
# =========================
DETECT_MODEL = "hog"         # "hog" hoặc "cnn"
LANDMARKS_MODEL = "large"    # "small" hoặc "large"
UPSAMPLE = 0                # 0 nhanh, tăng lên (1,2) bắt mặt nhỏ tốt hơn nhưng chậm

In [40]:
# =========================
# TIỀN XỬ LÝ ẢNH / ENCODING
# =========================
def _box_area(box: Tuple[int, int, int, int]) -> int:
    """Tính diện tích box (top, right, bottom, left)."""
    top, right, bottom, left = box
    return max(0, bottom - top) * max(0, right - left)


def encode_image(
    image_path: str,
    detect_model: str = DETECT_MODEL,
    landmarks_model: str = LANDMARKS_MODEL,
    upsample: int = UPSAMPLE,
    choose_largest_face: bool = True
) -> Optional[np.ndarray]:
    """
    Đọc ảnh và trả về encoding đầu tiên (hoặc mặt lớn nhất nếu choose_largest_face=True).

    Pipeline:
    - Kiểm tra file
    - Load ảnh
    - Detect face locations (hog/cnn)
    - (Tuỳ chọn) chọn 1 mặt lớn nhất nếu có nhiều mặt
    - Face encodings (embedding 128D) với landmarks model (small/large)
    """
    if not os.path.isfile(image_path):
        print(f"[BỎ QUA] Không tìm thấy ảnh: {image_path}")
        return None

    try:
        image = face_recognition.load_image_file(image_path)
    except Exception as e:
        print(f"[BỎ QUA] Lỗi đọc ảnh: {image_path} | {e}")
        return None

    # 1) Detect mặt (bounding boxes)
    try:
        locs = face_recognition.face_locations(
            image,
            number_of_times_to_upsample=upsample,
            model=detect_model
        )
    except Exception as e:
        print(f"[BỎ QUA] Lỗi detect face: {image_path} | {e}")
        return None

    if not locs:
        print(f"[BỎ QUA] Không tìm thấy khuôn mặt trong ảnh: {image_path}")
        return None

    # 2) Nếu có nhiều mặt: chọn mặt lớn nhất (thường là chủ thể)
    if choose_largest_face and len(locs) > 1:
        locs = [max(locs, key=_box_area)]

    # 3) Tạo embedding 128D dựa trên locations đã biết
    try:
        encs = face_recognition.face_encodings(
            image,
            known_face_locations=locs,
            model=landmarks_model  # "small" hoặc "large"
        )
    except Exception as e:
        print(f"[BỎ QUA] Lỗi tạo encodings: {image_path} | {e}")
        return None

    if not encs:
        print(f"[BỎ QUA] Không tạo được encoding cho ảnh: {image_path}")
        return None

    return encs[0]

In [41]:
# =========================
# LOAD CẶP ẢNH
# =========================
def read_pairs(pair_csv: Path, mismatch: bool = False):
    """Đọc file pairs. Trả về list tuple (path1, path2, is_match)."""
    pairs = []
    with open(pair_csv, newline="", encoding="utf-8") as f:
        reader = csv.reader(f)

        # Đọc dòng đầu, nếu là header thì bỏ qua, không thì xử lý như dữ liệu
        first = next(reader, None)
        if first is None:
            return pairs

        def is_data_row(row: List[str]) -> bool:
            try:
                if mismatch:
                    if len(row) != 4:
                        return False
                    int(row[1])
                    int(row[3])
                    return True
                if len(row) != 3:
                    return False
                int(row[1])
                int(row[2])
                return True
            except (TypeError, ValueError):
                return False

        rows_iter = reader
        if is_data_row(first):
            # first là dữ liệu
            rows_iter = [first] + list(reader)

        for row in rows_iter:
            if mismatch:
                if len(row) != 4:
                    continue
                name1, num1, name2, num2 = row
            else:
                if len(row) != 3:
                    continue
                name1, num1, num2 = row
                name2 = name1

            img1 = str(LFW_DIR / name1 / f"{name1}_{int(num1):04d}.jpg")
            img2 = str(LFW_DIR / name2 / f"{name2}_{int(num2):04d}.jpg")
            pairs.append((img1, img2, not mismatch))

    return pairs

In [42]:
# =========================
# ĐÁNH GIÁ VERIFICATION (CẶP ẢNH)
# =========================
def evaluate_pairs(threshold: float = 0.6, limit: Optional[int] = None):
    """Đánh giá theo nhiệm vụ verification (so khớp cặp ảnh) + cache encodings để chạy nhanh."""
    match_csv = DATA_DIR / "matchpairsDevTest.csv"
    mismatch_csv = DATA_DIR / "mismatchpairsDevTest.csv"
    match_pairs = read_pairs(match_csv, mismatch=False)
    mismatch_pairs = read_pairs(mismatch_csv, mismatch=True)

    if limit is not None and limit > 0:
        match_pairs = match_pairs[:limit]
        mismatch_pairs = mismatch_pairs[:limit]

    pairs = match_pairs + mismatch_pairs

    enc_cache: Dict[str, Optional[np.ndarray]] = {}

    def get_enc(path: str) -> Optional[np.ndarray]:
        if path not in enc_cache:
            enc_cache[path] = encode_image(path)
        return enc_cache[path]

    total = 0
    correct = 0
    skipped = 0

    for img1, img2, is_match in pairs:
        enc1 = get_enc(img1)
        enc2 = get_enc(img2)
        if enc1 is None or enc2 is None:
            skipped += 1
            continue

        dist = float(np.linalg.norm(enc1 - enc2))
        pred_match = dist < threshold
        if pred_match == is_match:
            correct += 1
        total += 1

    if total == 0:
        print("[EVAL-PAIRS] Không đủ dữ liệu để đánh giá.")
        return

    acc = correct / total
    print(f"[EVAL-PAIRS] Số cặp match: {len(match_pairs)}, mismatch: {len(mismatch_pairs)}")
    print(f"[EVAL-PAIRS] Tổng cặp dùng được: {total}, bỏ qua: {skipped}")
    print(f"[EVAL-PAIRS] Ngưỡng {threshold:.2f} | Accuracy: {acc:.4f}")


In [43]:
threshold = 0.6
limit = None

In [44]:
evaluate_pairs(threshold, limit)

[BỎ QUA] Không tìm thấy khuôn mặt trong ảnh: C:\Users\PC-09\Documents\Projects\face_recognition_svm\data\lfw-deepfunneled\lfw-deepfunneled\Amelie_Mauresmo\Amelie_Mauresmo_0021.jpg
[BỎ QUA] Không tìm thấy khuôn mặt trong ảnh: C:\Users\PC-09\Documents\Projects\face_recognition_svm\data\lfw-deepfunneled\lfw-deepfunneled\Elisabeth_Schumacher\Elisabeth_Schumacher_0001.jpg
[BỎ QUA] Không tìm thấy khuôn mặt trong ảnh: C:\Users\PC-09\Documents\Projects\face_recognition_svm\data\lfw-deepfunneled\lfw-deepfunneled\Luis_Horna\Luis_Horna_0002.jpg
[BỎ QUA] Không tìm thấy khuôn mặt trong ảnh: C:\Users\PC-09\Documents\Projects\face_recognition_svm\data\lfw-deepfunneled\lfw-deepfunneled\Zach_Safrin\Zach_Safrin_0001.jpg
[BỎ QUA] Không tìm thấy khuôn mặt trong ảnh: C:\Users\PC-09\Documents\Projects\face_recognition_svm\data\lfw-deepfunneled\lfw-deepfunneled\Damarius_Bilbo\Damarius_Bilbo_0001.jpg
[BỎ QUA] Không tìm thấy khuôn mặt trong ảnh: C:\Users\PC-09\Documents\Projects\face_recognition_svm\data\lfw-d